In [1]:
import model as m
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os

os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=256)])

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')

    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2022-03-25 10:12:22.614528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 10:12:22.728538: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 10:12:22.729058: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 10:12:22.736192: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 10:12:22.736718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [2]:
model= m.NN_Model()
model.get_data("laser_log/laser_log2203w.csv")

beta=[0.4]
drop=[0.6]
epochs=50
batch_size=16

group_epochs=1

results=[]
min_pos_error=0.3

for i in beta:
    for j in drop:
        #with sess.as_default():
            model.model_define(drop_rate=j)
            model.set_beta(i)
            model.model_compile("adam",model.pos_loss,['mean_absolute_error'])
            for step in range(epochs//group_epochs):
                history = model.model_run(group_epochs,batch_size, verbose=1)
                pos_mse,rot_mse,pos_mae,rot_mae = model.get_error(verbose=1)
                results.append({"beta":i,"drop-out":j,"epochs":(step+1)*group_epochs,"history":history,"position squared error":pos_mse, "rotation squared error":rot_mse, "position absolute error":pos_mae, "rotation absolute error":rot_mae})  
                if pos_mae < min_pos_error:
                    model.model_cnn.save_weights("models/cp-epoch-train.ckpt")
                    min_pos_error=pos_mae

2022-03-25 10:12:58.164136: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


52/52 [==============================] - 11s 74ms/step - loss: 9.4032 - mean_absolute_error: 7.1370 - val_loss: 0.8152 - val_mean_absolute_error: 0.5963
position mean square error: 0.7564465324874305 -- rotation mean square error: 1.0124634587112136 -- position mean absolute error: 0.5799864187716061 -- rotation mean absolute error: 0.6289680460734026
52/52 [==============================] - 3s 67ms/step - loss: 0.8453 - mean_absolute_error: 0.6222 - val_loss: 0.7510 - val_mean_absolute_error: 0.5472
position mean square error: 0.49782688253471563 -- rotation mean square error: 0.6658046475205062 -- position mean absolute error: 0.5246720079945828 -- rotation mean absolute error: 0.592117015526441
52/52 [==============================] - 4s 68ms/step - loss: 0.6447 - mean_absolute_error: 0.4778 - val_loss: 0.8001 - val_mean_absolute_error: 0.6021
position mean square error: 0.5198928673713012 -- rotation mean square error: 0.5993536725254348 -- position mean absolute error: 0.609456466

2022-03-25 10:13:34.193092: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-03-25 10:13:34.342004: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-03-25 10:13:34.482514: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.


52/52 [==============================] - 4s 73ms/step - loss: 0.4118 - mean_absolute_error: 0.3045 - val_loss: 0.7335 - val_mean_absolute_error: 0.5461
position mean square error: 0.5401097660628893 -- rotation mean square error: 0.22302196724733514 -- position mean absolute error: 0.6310390727912516 -- rotation mean absolute error: 0.3761027844730972
52/52 [==============================] - 4s 74ms/step - loss: 0.4470 - mean_absolute_error: 0.3298 - val_loss: 0.6412 - val_mean_absolute_error: 0.4625
position mean square error: 0.45081524858287875 -- rotation mean square error: 0.15466701592073914 -- position mean absolute error: 0.5428628335768936 -- rotation mean absolute error: 0.3018844872281316
52/52 [==============================] - 4s 70ms/step - loss: 0.4205 - mean_absolute_error: 0.3116 - val_loss: 0.3973 - val_mean_absolute_error: 0.2972
position mean square error: 0.11950850069894997 -- rotation mean square error: 0.21616370842335825 -- position mean absolute error: 0.25590

2022-03-25 10:13:52.246178: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-03-25 10:13:52.372117: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.


52/52 [==============================] - 4s 73ms/step - loss: 0.3563 - mean_absolute_error: 0.2617 - val_loss: 0.3125 - val_mean_absolute_error: 0.2295
position mean square error: 0.11388090356409133 -- rotation mean square error: 0.08145863281129095 -- position mean absolute error: 0.25336772720088124 -- rotation mean absolute error: 0.18167800316095378
52/52 [==============================] - 4s 75ms/step - loss: 0.3767 - mean_absolute_error: 0.2789 - val_loss: 0.4779 - val_mean_absolute_error: 0.3391
position mean square error: 0.2432179335880572 -- rotation mean square error: 0.15143067022914947 -- position mean absolute error: 0.37095238325796265 -- rotation mean absolute error: 0.27548979995443496
52/52 [==============================] - 4s 70ms/step - loss: 0.3356 - mean_absolute_error: 0.2459 - val_loss: 0.3080 - val_mean_absolute_error: 0.2287
position mean square error: 0.09091426106470894 -- rotation mean square error: 0.09555012898052265 -- position mean absolute error: 0.2

In [4]:
with open('test46k.csv', 'w') as f:
    f.write("beta;dropout;epochs;pos_mse;pos_mae;rot_mse;rot_mae\n")
    for item in results:
        f.write("{};{};{};{};{};{};{}\n".format(item["beta"],item["drop-out"],item["epochs"],item["position squared error"],item["position absolute error"],item["rotation squared error"],item["rotation absolute error"]))

#Valutation

In [3]:
model.model_cnn.load_weights("models/cp-epoch-train.ckpt")

2022-03-23 14:17:23.462731: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/cp-epoch-train.ckpt: FAILED_PRECONDITION: models/cp-epoch-train.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [4]:
data_test = m.DAO("laser_log/laser_log2203w.csv")

data_test.read()
data_test.divide_data(0.25)

data_tensor_test=m.DAO()

data_tensor_test.TR_features=tf.reshape(data_test.TR_features,[-1,720,1,1])
data_tensor_test.TS_features=tf.reshape(data_test.TS_features,[-1,720,1,1])
data_tensor_test.TR_targets=tf.reshape(data_test.TR_targets,[-1,3])
data_tensor_test.TS_targets=tf.reshape(data_test.TS_targets,[-1,3])



In [26]:
result = model.model_cnn.predict(data_tensor_test.TS_features)
pos_test = result[:,0:2]
rot_test = result[:,2]
pos_real = np.array([data_test.TS_targets["pos_x"].to_list(),data_test.TS_targets["pos_y"].to_list()]).T
rot_real = (data_test.TS_targets["pos_yaw"]).to_numpy()

In [24]:
pos_mae = mean_squared_error(pos_real,pos_test)
rot_mae = mean_absolute_error(rot_real,rot_test)

In [27]:
with open('predict_data2.csv', 'w') as f:
    for i in range(len(rot_real)):
        f.write("{};{};{};{};{};{}\n".format(pos_real[i][0],pos_real[i][1],rot_real[i],pos_test[i][0],pos_test[i][1],rot_test[i]))